In [38]:
from itertools import product
from random import random, randint, choice, seed
import numpy as np
from scipy import sparse
from copy import copy


In [39]:


def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets



# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [40]:
num_points=100
num_sets=num_points
x = make_set_covering_problem(num_points, num_sets, .3)
counter=0
print("Element at row=42 and column=42:", x[42,42])

Element at row=42 and column=42: True


In [45]:


def tweak(state):
    new_state = copy(state)
    index = randint(0, num_sets - 1)
    new_state[index] = not new_state[index]
    return new_state



def fitness_function(num_points, num_sets, density):
    sets = make_set_covering_problem(num_points, num_sets, density)
    total_sets = np.sum(sets, axis=0)
    fitness = np.sum((total_sets - np.min(total_sets))**2)
    return fitness

def fitness(state):
    global counter
    counter += 1

    # Calculate cost (number of selected sets)
    cost = sum(state)

    # Initialize support matrices
    l = np.zeros((num_sets, num_points), dtype=bool)
    not_taken = np.zeros((num_sets, num_points), dtype=bool)

    # Populate support matrices
    for j in range(num_sets):
        for i in range(num_points):
            if state[j]:
                l[j, i] = x[j, i]
            else:
                not_taken[j, i] = x[j, i]

    # Calculate elements already covered by the selected sets
    already_covered = np.any(l[state, :], axis=0)

    # Calculate the number of elements already covered
    valid = np.sum(already_covered)

    # Evaluate fitness
    fitness_value = (valid, -cost) if valid > 0 else (0, 0)

    return fitness_value






In [44]:
def hill_climbing():
    global counter
    counter = 0
    
    # Initialize a random initial state
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    
    # Flag to indicate the end of the algorithm
    ended = False
    
    # Counter for the number of iterations without improvement
    not_improving = 0
    
    while not ended or not_improving > 100:
        # Generate the next state by applying a small tweak
        new_state = tweak(current_state)

        # Calculate the fitness of the next state
        new_fitness = fitness(new_state)
        
        # Get the number of points covered by the next state
        new_covered_points = new_fitness[0]

        # If all points are covered, terminate the algorithm
        if new_covered_points == num_points:
            ended = True

        # Check if the next state is better in terms of fitness
        is_better = new_fitness > fitness(current_state)

        # If the next state is better, update the current state
        if is_better:
            not_improving = 0  # Reset the counter for iterations without improvement
            current_state = new_state
        else:
            not_improving += 1  # Increment the counter for iterations without improvement

        counter += 1  # Increment the global counter for total iterations

    # Print the number of steps needed to solve the problem
    print(f"Solved in {counter:,} steps")
    
    # Print the final solution
    print("Final solution:", fitness(current_state))

# Call the hill_climbing function
hill_climbing()


Solved in 36 steps
Final solution: (100, -11)


In [50]:
def hill_climbing_it(current_state): 
    
    ended=False
    is_better=True
    not_improving=0

    while (not ended) or not_improving>100:
       
        new_state = tweak(current_state)

        new_f=fitness(new_state)
        
        new_covered_points=new_f[0]

        if new_covered_points==num_points:
            ended=True

        is_better=new_f>fitness(current_state)

        if is_better:
            not_improving=not_improving-1
            current_state = new_state
        else:
            not_improving=not_improving+1
    
    return current_state

def iterated_local_search():
    global counter
    counter = 0
    
    # Initialize the current state and the best solution
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    best_solution = current_state
   
    # Iterate for a fixed number of times (5 iterations in this case)
    for iteration in range(5):
        print("Iteration:", iteration)
        
        # Apply Iterated Local Search using hill climbing with steepest step
        current_solution = hill_climbing_it(tweak(best_solution))
    
        # Update the best solution if the current solution is better
        if fitness(current_solution) > fitness(best_solution):
            best_solution = current_solution
           
        # Print the fitness of the current best solution after each iteration
        print("Current Best Fitness:", fitness(best_solution))
    
    # Print the final result
    print(f"Solved in {counter:,} steps")
    print("Final solution:", fitness(best_solution))

# Call the iterated_local_search function
iterated_local_search()


Iteration: 0
Current Best Fitness: (100, -12)
Iteration: 1
Current Best Fitness: (100, -12)
Iteration: 2
Current Best Fitness: (100, -12)
Iteration: 3
Current Best Fitness: (100, -12)
Iteration: 4
Current Best Fitness: (100, -12)
Solved in 59 steps
Final solution: (100, -12)


TABU SEARCH

In [49]:
def is_valid(sol, current_state):
    # Check if the solution has at least one selected set
    return np.sum(sol) > 0

def tabu_search():
    global counter
    counter = 0
    
    # Initialize the tabu list and current state
    tabu_list = []
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    
    # Main loop for a fixed number of steps (100 in this case)
    for step in range(100):   
        print("Step:", step)
        
        # Generate a set of candidate solutions using the tweak function
        candidate_solutions = (tweak(current_state) for _ in range(num_points))
        
        # Filter candidates that are valid and not in the tabu list
        valid_candidates = [(sol, fitness(sol)) for sol in candidate_solutions if is_valid(sol, current_state) and sol not in tabu_list]
        
        # Check if there are valid candidates
        if not valid_candidates:
            continue
        else:
            # Select the candidate with the maximum fitness value
            best_candidate = max(valid_candidates, key=lambda x: x[1])

            # Update the current state if the new solution is better
            if fitness(best_candidate[0]) > fitness(current_state):
                current_state = best_candidate[0]
                
            # Add the current state to the tabu list
            tabu_list.append(current_state)
   
    # Choose the best solution from the tabu list based on fitness
    best_solution = max(tabu_list, key=lambda x: fitness(x))
    
    # Print the result
    print(f"Solved in {counter:,} evaluations")
    print("Best solution:", fitness(best_solution))

# Call the tabu_search function
tabu_search()


Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Solved in 10,217 evaluations
Best solution: (100, -6)
